pandas-ylt
==========

Python modules for working in pandas with various loss data tables used for CAT modelling


### Read the example YLT

In [1]:
import pandas as pd
from pandas_ylt.yeareventloss import YearEventLossTable
from pandas_ylt.yeareventallocloss import YearEventAllocLossTable


yelt = pd.read_csv("tests/_data/example_pareto_poisson_yelt.csv",
                    index_col=['Year', 'EventID', 'DayOfYear'])['Loss']
yelt.attrs['n_yrs'] = 1e5

display(yelt.yel.to_ep_summaries([200, 100, 10]).style.format("{:,.0f}"))
display(yelt.yel.to_aal_series().to_frame().style.format("{:,.0f}"))

CurveType,LossAEP,LossOEP
ReturnPeriod,,
200,"104,018","101,450"
100,"51,611","49,340"
10,"5,539","4,708"


,
AAL,"5,564"
STD,"158,881"


### Define some layers

In [2]:
# Define the layers
from pandas_ylt.layer import Layer


layers = [
    # First loss
    {'name': '20 xs 0', 'layer': Layer(**{'limit': 20000, 'xs': 0, 'agg_limit': 20000.0})},    
    {'name': '20 xs 20', 'layer': Layer(**{'limit': 20000, 'xs': 20000, 'agg_limit': 20000.0})},
    {'name': '20 xs 40', 'layer': Layer(**{'limit': 20000, 'xs': 40000, 'agg_limit': 20000.0})},
    {'name': '20 xs 60', 'layer': Layer(**{'limit': 20000, 'xs': 60000, 'agg_limit': 20000.0})},    
    {'name': '20 xs 80', 'layer': Layer(**{'limit': 20000, 'xs': 80000, 'agg_limit': 20000.0})},    
    # Second loss
    {'name': '20 xs 0 reinst', 'layer': Layer(**{'limit': 20000, 'xs': 0, 'agg_limit': 20000.0, 'agg_xs': 20000.0})},        
    {'name': '20 xs 20 reinst', 'layer': Layer(**{'limit': 20000, 'xs': 20000, 'agg_limit': 20000.0, 'agg_xs': 20000.0})},
    {'name': '20 xs 40 reinst', 'layer': Layer(**{'limit': 20000, 'xs': 40000, 'agg_limit': 20000.0, 'agg_xs': 20000.0})},
    {'name': '20 xs 60 reinst', 'layer': Layer(**{'limit': 20000, 'xs': 60000, 'agg_limit': 20000.0, 'agg_xs': 20000.0})}, 
    {'name': '20 xs 80 reinst', 'layer': Layer(**{'limit': 20000, 'xs': 80000, 'agg_limit': 20000.0, 'agg_xs': 20000.0})}, 
]



### Calculate the layer loss

In [3]:
lyr_yelts = pd.concat([lyr['layer'].yelt_loss(yelt) for lyr in layers], 
                      keys=[lyr['name'] for lyr in layers], names=['Layer'])
lyr_yelts

Layer            Year   EventID  DayOfYear
20 xs 0          698    23302    1            17373.822263
                 1019   30835    1             3473.484522
                 1162   13431    1            20000.000000
                 2322   18394    1            17957.088848
                 2925   38164    1             1122.213901
                                                  ...     
20 xs 40 reinst  85576  6288     353           6353.328397
                 72621  21042    365          11313.472407
20 xs 60 reinst  26941  5978     252           2278.453096
                 62337  23404    278          20000.000000
20 xs 80 reinst  62337  23404    278           8905.218687
Name: Loss, Length: 56340, dtype: float64

### Get the AEP and OEP

In [4]:
ceded_yelt = lyr_yelts.groupby(yelt.index.names).sum()
retained_yelt = yelt.subtract(ceded_yelt, fill_value=0.0)

return_periods = [500, 250, 200, 150, 100, 90, 80, 70, 60, 50, 40, 30, 20, 10]
ep0 = yelt.yel.to_ep_summaries(return_periods)
ep1 = ceded_yelt.yel.to_ep_summaries(return_periods)
ep2 = retained_yelt.yel.to_ep_summaries(return_periods)

pd.concat([ep0, ep1, ep2], axis=1, keys=['Gross', 'Ceded', 'Retained']).style.format("{:,.0f}")

### Get the AAL per layer

In [5]:
for lyr_name, grp in lyr_yelts.groupby('Layer'):
    print(f"{lyr_name:>20}: {grp.yel.aal:,.0f}")

print(f"{'Retained':>20s}: {retained_yelt.yel.aal:,.0f}")

             20 xs 0: 1,934
      20 xs 0 reinst: 49
            20 xs 20: 334
     20 xs 20 reinst: 2
            20 xs 40: 200
     20 xs 40 reinst: 1
            20 xs 60: 144
     20 xs 60 reinst: 0
            20 xs 80: 114
     20 xs 80 reinst: 0
            Retained: 2,785
